# ABX Test with webMUSHRA


Below we create yaml config-file for an ABX-test using [webMUSHRA](https://github.com/audiolabs/webMUSHRA)


In [ ]:
import numpy as np  # needed for random

In [ ]:
Trials = 4  # must be even integer for below handling
num_ones = 0  # use this if ABX reference should always be 'treatment1'
num_ones = Trials  # use this if ABX reference should always be 'treatment2'
num_ones = Trials//2  # 'treatment1' or 'treatment 2' as reference by 50:50 chance  

audio = ["pnoise"]
treatment1 = ["ref"]
treatment2 = ["treat"]

part = 0  # starting from 0

filename = "ABX_SchultzHahn19_Training"
testId = "abx_training"
testname = "Herzlich Willkommen zum ABX-Test Training"
allRandomized = True
showResults = "true"
writeResults = "false"
showWaveform = "false"
enableLooping = "false"

In [ ]:
## don't touch the stuff below, unless you know what's going on
if (len(audio) == len(treatment1) == len(treatment2)) == False:
    print("please check audio, treatment lists for proper assignment (length not equal)")
    
bufferSize = 4096
introtext = "Tastatur Start/Stop: 0...Stimulus X,  1...Stimulus A,  2...Stimulus B"
introcomment = "Wenn bereit, bitte auf ''Nächste Seite'' klicken."

f = open(filename+".yaml", "w+")
f2 = open(filename+".txt", "w+")

print("testname:", testname, file=f)
print("testId:", testId, file=f)
print("bufferSize:", bufferSize, file=f)
print("stopOnErrors: true", file=f)
print("showButtonPreviousPage: false", file=f)
print("remoteService: service/write.php", file=f)
print("pages:", file=f)

# intro dialog
ident = "  "  # tested only for 2 spaces
print(1*ident+"- type: generic", file=f)
print(1*ident+"  id: first_page", file=f)
print(1*ident+"  name: "+introtext, file=f)
print(1*ident+"  content: "+introcomment, file=f)

# prep random section for stimuli 
print(1*ident+"-", file=f)
print(2*ident+"- random", file=f)

#if allRandomized == False:
#    print(2*ident+"-", file=f)  

np.random.seed(1)  # make the yaml sequence itself fixed random
# truly randomized playback is ensured by the webMUSHRA API

# do for all audio & number of trials that are defined above:
cnt_all = 0  # total trials counter
for (a,t1,t2) in zip(audio, treatment1, treatment2):
    arr = np.zeros(Trials, dtype=int)
    idx = np.random.choice(range(Trials), num_ones, replace=False)
    arr[idx] = 1
    cnt = 0
    if allRandomized == False:  # trials per one audio in sequence, audio randomized 
        print(2*ident+"-", file=f)  
    for i in arr:
        if allRandomized == True:  # all trials in random sequence
            print(2*ident+"-", file=f)    
        if arr[i] == 0:
            phase_ref = t1
            phase_C = t2
        elif arr[i] == 1:
            phase_C = t1
            phase_ref = t2
        else:
            print("we never should get here")
        print(3*ident+"- type: paired_comparison", file=f)
        if cnt<10:
            print(3*ident+"  id: "+a+'_'+str(part)+"_0"+str(cnt), file=f)
        else:
            print(3*ident+"  id: "+a+'_'+str(part)+"_"+str(cnt), file=f)
        print("ID "+str(cnt)+", Trial: "+str(cnt_all)+"/"+str(len(audio) * Trials)+":\n"+a+str(cnt),file=f2)
        cnt+= 1  # for individual id
        cnt_all += 1 # counter for all stimuli
        print(3*ident+"  name: "+a, file=f)
        print(3*ident+"  unforced: false", file=f)
        print(3*ident+"  content: Ist A = X oder B = X ?", file=f)
        print(3*ident+"  showWaveform: "+showWaveform, file=f)
        print(3*ident+"  enableLooping: "+enableLooping, file=f)
        print(3*ident+"  reference: configs/resources/stimuli/"+a+"_"+phase_ref+".wav", file=f)
        print(3*ident+"  stimuli:", file=f)
        print(4*ident+"  C1: configs/resources/stimuli/"+a+"_"+phase_C+".wav", file=f)
        print("reference: "+a+"_"+phase_ref,file=f2)
        print("treatment: "+a+"_"+phase_C+"\n",file=f2)

# finish dialog
print(1*ident+"- type: finish", file=f)
print(1*ident+"  name: Training Fertig :-)", file=f)
print(1*ident+"  content: Herzlichen Dank!", file=f)
print(1*ident+"  showResults: "+showResults, file=f)
print(1*ident+"  writeResults: "+writeResults, file=f)

f.close()
f2.close()
#print("total trials :", cnt_all)